In [ ]:
# Aim: To simulate and animate the electric field and equipotential lines of an electric dipole in 2D and 3D space due to charges at user-defined coordinates.

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from typing import Tuple

# --- Constants ---
K_COULOMB = 8.98755e9  # Coulomb's constant in N·m²/C²

class TwoChargeSimulator:
    """
    A class to calculate and visualize the electric field and potential of two opposite point charges.
    """
    def __init__(self, charge_magnitude: float, pos_charge_coords: Tuple[float, float, float], neg_charge_coords: Tuple[float, float, float]):
        """
        Initializes the simulation with two charges at specified coordinates.

        Args:
            charge_magnitude (float): Magnitude of the positive charge (+q). The negative charge will be -q.
            pos_charge_coords (Tuple): The (x, y, z) coordinates of the positive charge.
            neg_charge_coords (Tuple): The (x, y, z) coordinates of the negative charge.
        """
        if charge_magnitude <= 0:
            raise ValueError("Charge magnitude must be positive.")
        self.q = charge_magnitude
        self.pos_charge_pos = np.array(pos_charge_coords, dtype=float)
        self.neg_charge_pos = np.array(neg_charge_coords, dtype=float)

    def calculate_field_and_potential(self, x: np.ndarray, y: np.ndarray, z: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Calculates the electric field (Ex, Ey, Ez) and potential (V) on a 3D grid.
        """
        r_pos = np.stack([x - self.pos_charge_pos[0], y - self.pos_charge_pos[1], z - self.pos_charge_pos[2]], axis=-1)
        r_neg = np.stack([x - self.neg_charge_pos[0], y - self.neg_charge_pos[1], z - self.neg_charge_pos[2]], axis=-1)

        r_mag_pos = np.linalg.norm(r_pos, axis=-1) + 1e-9
        r_mag_neg = np.linalg.norm(r_neg, axis=-1) + 1e-9

        v_total = (K_COULOMB * self.q / r_mag_pos) + (K_COULOMB * -self.q / r_mag_neg)
        e_total = (K_COULOMB * self.q * r_pos / r_mag_pos[..., np.newaxis]**3) + (K_COULOMB * -self.q * r_neg / r_mag_neg[..., np.newaxis]**3)
        
        return e_total[..., 0], e_total[..., 1], e_total[..., 2], v_total

    def plot_2d(self, grid_size: float = 1.0, grid_points: int = 50):
        """
        Generates a static 2D plot of the electric field and equipotential lines.
        """
        x_vals = np.linspace(-grid_size, grid_size, grid_points)
        y_vals = np.linspace(-grid_size, grid_size, grid_points)
        X, Y = np.meshgrid(x_vals, y_vals)
        Z = np.zeros_like(X)

        Ex, Ey, _, V = self.calculate_field_and_potential(X, Y, Z)
        
        plt.style.use('seaborn-v0_8-darkgrid')
        fig, ax = plt.subplots(figsize=(10, 8))

        v_abs_max = np.max(np.abs(V))
        contour_levels = np.linspace(-v_abs_max * 0.8, v_abs_max * 0.8, 30)
        cont = ax.contour(X, Y, V, levels=contour_levels, cmap='RdBu', linewidths=1.0)
        fig.colorbar(cont, label='Electric Potential (V)')

        ax.streamplot(X, Y, Ex, Ey, color='black', linewidth=0.7, density=1.5, arrowstyle='->', arrowsize=1.2)
        ax.plot(self.pos_charge_pos[0], self.pos_charge_pos[1], 'o', markersize=12, color='red', label='+q')
        ax.plot(self.neg_charge_pos[0], self.neg_charge_pos[1], 'o', markersize=12, color='blue', label='-q')

        ax.set_title(f'Electric Field of Two Charges (q={self.q:.2e} C)', fontsize=16)
        ax.set_xlabel('X (m)'); ax.set_ylabel('Y (m)')
        ax.set_aspect('equal', adjustable='box')
        ax.legend()
        plt.show()

    def animate_2d_movement(self, grid_size: float = 1.0, grid_points: int = 40, frames: int = 120, interval: int = 50):
        """
        Generates a 2D animation showing one charge oscillating. 🧬
        """
        fig, ax = plt.subplots(figsize=(10, 8))
        x_vals = np.linspace(-grid_size, grid_size, grid_points)
        y_vals = np.linspace(-grid_size, grid_points, grid_points)
        X, Y = np.meshgrid(x_vals, y_vals)
        Z = np.zeros_like(X)
        
        # Store initial positions to reset for the animation logic
        initial_pos_charge_pos = self.pos_charge_pos.copy()
        initial_neg_charge_pos = self.neg_charge_pos.copy()

        def update(frame):
            ax.clear()
            # Animate the positive charge oscillating vertically
            oscillation = 0.2 * grid_size * np.sin(2 * np.pi * frame / frames)
            self.pos_charge_pos = initial_pos_charge_pos + np.array([0, oscillation, 0])
            # Keep the negative charge fixed
            self.neg_charge_pos = initial_neg_charge_pos

            Ex, Ey, _, V = self.calculate_field_and_potential(X, Y, Z)
            
            # Plotting
            v_abs_max = K_COULOMB * self.q / (0.1) # Estimate a fixed potential range
            contour_levels = np.linspace(-v_abs_max, v_abs_max, 25)
            ax.contour(X, Y, V, levels=contour_levels, cmap='RdBu', linewidths=1.0)
            ax.streamplot(X, Y, Ex, Ey, color='black', linewidth=0.7, density=1.5)
            ax.plot(self.pos_charge_pos[0], self.pos_charge_pos[1], 'o', markersize=12, color='red', label='+q')
            ax.plot(self.neg_charge_pos[0], self.neg_charge_pos[1], 'o', markersize=12, color='blue', label='-q')
            
            # Formatting
            ax.set_title(f'Field with Oscillating Charge (+q)', fontsize=16)
            ax.set_xlabel('X (m)'); ax.set_ylabel('Y (m)')
            ax.set_xlim(-grid_size, grid_size); ax.set_ylim(-grid_size, grid_size)
            ax.set_aspect('equal', adjustable='box')
            ax.legend(loc='upper right')

        ani = FuncAnimation(fig, update, frames=frames, interval=interval, blit=False)
        plt.show()


if __name__ == '__main__':
    # --- User-defined parameters ---
    charge_val = 12e-9  # Charge magnitude in Coulombs (1 nC)
    
    # Define custom coordinates for the charges
    positive_charge_coords = (-0.02, -0.02, 0)
    negative_charge_coords = (0.02, -0.02, 0)

    # --- 1. Generate a STATIC plot with the custom coordinates ---
    print("Generating 2D static plot with charges at custom coordinates...")
    static_sim = TwoChargeSimulator(
        charge_magnitude=charge_val,
        pos_charge_coords=positive_charge_coords,
        neg_charge_coords=negative_charge_coords
    )
    static_sim.plot_2d(grid_size=0.5, grid_points=50)

    # --- 2. Generate an ANIMATED plot with an oscillating charge ---
    print("\nGenerating 2D animation with one charge oscillating...")
    # For the animation, let's start with a standard horizontal dipole
    animation_sim = TwoChargeSimulator(
        charge_magnitude=charge_val,
        pos_charge_coords=(0.15, 0, 0),
        neg_charge_coords=(-0.15, 0, 0)
    )
    animation_sim.animate_2d_movement(grid_size=0.6, frames=100, interval=50)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- Physics Functions ---

def get_potential(X, Y, charges):
    """
    Calculates the electric potential V on a 2D grid (X, Y).
    charges is a list of tuples: [(q1, (x1, y1)), (q2, (x2, y2)), ...]
    We assume k = 1 / (4*pi*epsilon_0) = 1 for simplicity.
    """
    V = np.zeros_like(X)
    for q, pos in charges:
        # Calculate distance from each charge
        # Add a small epsilon (1e-9) to avoid division by zero at the charge's location
        dx = X - pos[0]
        dy = Y - pos[1]
        r = np.sqrt(dx**2 + dy**2) + 1e-9
        
        # V = kq / r
        V += q / r
    return V

def get_field(X, Y, charges):
    """
    Calculates the electric field (Ex, Ey) on a 2D grid (X, Y).
    E = -grad(V) or E = sum(k*q*r_hat / r^2)
    Using E = k*q * r_vec / r^3
    """
    Ex = np.zeros_like(X)
    Ey = np.zeros_like(X)
    
    for q, pos in charges:
        dx = X - pos[0]
        dy = Y - pos[1]
        
        # Add epsilon to r^3 to avoid singularity
        r_cubed = (dx**2 + dy**2 + 1e-9)**1.5
        
        Ex += q * dx / r_cubed
        Ey += q * dy / r_cubed
        
    return Ex, Ey

# --- Simulation & Plotting Setup ---

# Set up the simulation grid
grid_size = 100
grid_lim = 5
x = np.linspace(-grid_lim, grid_lim, grid_size)
y = np.linspace(-grid_lim, grid_lim, grid_size)
X, Y = np.meshgrid(x, y)

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 8))

# --- Customization Parameters ---
# You can change these values
initial_charge_pos_1 = (1.5, 0)
initial_charge_pos_2 = (-1.5, 0)
q1 = 1.0   # Positive charge
q2 = -1.0  # Negative charge

# Animation parameters
total_frames = 200  # Total frames for the animation
rotation_speed = 0.05 # Radians per frame (controls speed of rotation)


def animate(frame):
    """
    This function is called for each frame of the animation.
    """
    ax.clear()  # Clear the previous frame
    
    # --- 1. Update Charge Positions (Rotating Dipole) ---
    # You can change this logic to any movement you want.
    angle = frame * rotation_speed
    
    r = np.sqrt(initial_charge_pos_1[0]**2 + initial_charge_pos_1[1]**2)
    
    x1 = r * np.cos(angle)
    y1 = r * np.sin(angle)
    
    x2 = -x1
    y2 = -y1
    
    charges = [
        (q1, (x1, y1)),
        (q2, (x2, y2))
    ]

    # --- 2. Calculate V and E ---
    V = get_potential(X, Y, charges)
    Ex, Ey = get_field(X, Y, charges)

    # --- 3. Plotting ---
    
    # Plot Equipotential Lines (Contour)
    # Define contour levels. Clipping V to avoid extreme values near the charges.
    v_min = -2
    v_max = 2
    levels = np.linspace(v_min, v_max, 40)
    V_clipped = np.clip(V, v_min, v_max)
    
    cont = ax.contour(X, Y, V_clipped, levels=levels, colors='gray', 
                      linewidths=0.7, linestyles='--')
    ax.clabel(cont, inline=True, fontsize=8, fmt='%.1f')

    # Plot Electric Field Lines (Streamplot)
    # Color the field lines by their magnitude
    E_mag = np.sqrt(Ex**2 + Ey**2)
    # Use log scale for color to see weaker fields better
    color = np.log(E_mag) 
    
    ax.streamplot(X, Y, Ex, Ey, color=color, linewidth=1, 
                  cmap='plasma', density=1.5, arrowstyle='->', arrowsize=1.2)

    # Plot the charges
    ax.plot(x1, y1, 'o', markersize=12, color='red', label='+q')
    ax.plot(x2, y2, 'o', markersize=12, color='blue', label='-q')
    
    # --- 4. Formatting ---
    ax.set_title(f'Animated Electric Dipole (Frame {frame})')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_xlim(-grid_lim, grid_lim)
    ax.set_ylim(-grid_lim, grid_lim)
    ax.set_aspect('equal')
    ax.legend()

# --- Run the Animation ---
# interval=20 means 20ms per frame (controls animation speed)
ani = animation.FuncAnimation(fig, animate, frames=total_frames, 
                              interval=20, blit=False)

plt.show()

# To save the animation (requires ffmpeg or imagemin):
# ani.save('dipole_animation.gif', writer='imagemagick', fps=30)
# ani.save('dipole_animation.mp4', writer='ffmpeg', fps=30)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_user_charges():
    """
    Asks the user to input all charge values and their positions.
    Returns a list of tuples: [(q1, x1), (q2, x2), ...]
    """
    charges = []
    
    while True:
        try:
            num_charges = int(input("How many charges do you want to define? "))
            if num_charges <= 0:
                print("Please enter a positive number.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter an integer.")

    print("\n--- Enter Charge Details ---")
    print("Example charge: 1.6e-19 (for a proton)")
    print("Example position: 0.05 (for 5 cm)")
    
    for i in range(num_charges):
        while True:
            try:
                q_str = input(f"  Charge {i+1} value (in Coulombs): ")
                q = float(q_str)
                
                x_str = input(f"  Charge {i+1} x-position (in meters): ")
                x = float(x_str)
                
                charges.append((q, x))
                print(f"  Added charge {q} C at x = {x} m\n")
                break
            except ValueError:
                print("Invalid input. Please use scientific notation (e.g., 1.6e-19) or decimals.")
                
    return charges

def get_plot_range():
    """Asks the user for the x-axis range to plot."""
    print("--- Enter Plot Range ---")
    while True:
        try:
            x_min = float(input("Enter x_min (meters): "))
            x_max = float(input("Enter x_max (meters): "))
            if x_min >= x_max:
                print("x_min must be less than x_max.")
                continue
            return x_min, x_max
        except ValueError:
            print("Invalid input. Please enter a number.")

def calculate_Ex_on_axis(x_points, charges, k=8.99e9):
    """
    Calculates the total E-field (Ex component) at all points in x_points.
    
    The vector form for the E-field from a single charge q_i at x_i is:
    E(x) = (k * q_i * (x - x_i)) / |x - x_i|^3
    This automatically handles the direction (sign).
    """
    Ex_total = np.zeros_like(x_points)
    
    for q, x_q in charges:
        # Calculate the vector from the charge (x_q) to the point (x)
        r_vec = x_points - x_q
        
        # Calculate the magnitude of r cubed
        r_mag_cubed = (np.abs(r_vec))**3
        
        # --- Handle the singularity (infinity) at the charge's location ---
        # Where r_mag_cubed is 0, set it to NaN (Not a Number)
        # This tells matplotlib to create a break in the line.
        r_mag_cubed[r_mag_cubed == 0] = np.nan
        
        # Calculate E for this charge and add it to the total
        # np.divide handles the NaNs gracefully
        Ex_i = k * q * np.divide(r_vec, r_mag_cubed)
        
        # Add to the total field. Note: (number + NaN) = NaN
        Ex_total += Ex_i
        
    return Ex_total

def plot_field(x_points, Ex_field, charges):
    """
    Plots the E-field vs. position and intelligently clips the y-axis
    to prevent infinities from making the plot unreadable.
    """
    plt.figure(figsize=(12, 7))
    
    # Plot the E-field
    plt.plot(x_points, Ex_field, 'b-', label='Total $E_x$ Field')
    
    # --- Smart Y-axis limiting ---
    # We use percentiles to clip the view, otherwise the
    # asymptotes (infinities) at the charges make the plot useless.
    
    # Get all field values that are NOT NaN
    clean_Ex = Ex_field[~np.isnan(Ex_field)] 
    
    if clean_Ex.size > 0:
        # Get the 1st and 99th percentile values
        v_min = np.percentile(clean_Ex, 1) 
        v_max = np.percentile(clean_Ex, 99)
        
        # Add 10% padding
        padding = (v_max - v_min) * 0.1
        plt.ylim(v_min - padding, v_max + padding)
        print(f"\nPlot Y-axis clipped to ({v_min - padding:.2e}, {v_max + padding:.2e}) for readability.")
    
    # --- Plot formatting ---
    
    # Draw vertical lines for charge locations
    for q, x_q in charges:
        color = 'red' if q > 0 else 'blue'
        label = f'Charge at x={x_q}m'
        plt.axvline(x=x_q, color=color, linestyle='--', label=label)

    plt.xlabel("Position x (m)", fontsize=12)
    plt.ylabel("Electric Field $E_x$ (N/C)", fontsize=12)
    plt.title("Electric Field vs. Position along the x-axis", fontsize=14)
    plt.grid(True, which='both', linestyle=':', linewidth=0.5)
    
    # Get unique labels for the legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    
    plt.show()

# --- Main Program ---
if __name__ == "__main__":
    print("--- 1D Electric Field Simulator ---")
    
    # 1. Get user inputs
    user_charges = get_user_charges()
    x_min, x_max = get_plot_range()
    
    # 2. Set up simulation grid
    # 2000 points gives a good resolution
    x_array = np.linspace(x_min, x_max, 2000)
    
    # 3. Run simulation
    print("\nCalculating field...")
    Ex_total = calculate_Ex_on_axis(x_array, user_charges)
    
    # 4. Plot results
    print("Generating plot...")
    plot_field(x_array, Ex_total, user_charges)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

# Use a style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')

# --- 1. User-Defined Charge Configuration ---
# The user can modify this list to add any charges.
# Format: [(charge_magnitude, (x, y, z)), ...]

# Default: A simple electric dipole
q = 1.0  # Magnitude of charge
d = 1.0  # Separation
charge_list = [
    (q, (0, d / 2, 0)),   # Positive charge
    (-q, (0, -d / 2, 0))  # Negative charge
]

# Example: A Quadrupole
# charge_list = [
#     (q, (d/2, d/2, 0)),
#     (-q, (-d/2, d/2, 0)),
#     (q, (-d/2, -d/2, 0)),
#     (-q, (d/2, -d/2, 0))
# ]

# --- 2. Core Physics Function ---

def get_E_grid(charges, X, Y, Z):
    """
    Calculates the electric field (Ex, Ey, Ez) on a 3D grid.
    
    Args:
        charges (list): A list of (q, (xq, yq, zq)) tuples.
        X (np.array): 3D grid of x-coordinates.
        Y (np.array): 3D grid of y-coordinates.
        Z (np.array): 3D grid of z-coordinates.
        
    Returns:
        (np.array, np.array, np.array): Ex, Ey, Ez components of the E-field.
    """
    # We'll use k=1 for simplicity in plotting the field shape
    k = 1 
    
    # Initialize electric field components
    Ex = np.zeros_like(X)
    Ey = np.zeros_like(Y)
    Ez = np.zeros_like(Z)
    
    # Superposition principle: loop over each charge
    for q, (xq, yq, zq) in charges:
        # Calculate vector components from charge to grid points
        Rx = X - xq
        Ry = Y - yq
        Rz = Z - zq
        
        # Calculate magnitude squared
        R_mag_sq = Rx**2 + Ry**2 + Rz**2
        
        # Avoid singularity at the charge's position
        R_mag_sq[R_mag_sq == 0] = 1e-6
        
        # E = k * q * R_vec / |R|^3
        R_mag_cubed = R_mag_sq**1.5
        
        Ex += k * q * Rx / R_mag_cubed
        Ey += k * q * Ry / R_mag_cubed
        Ez += k * q * Rz / R_mag_cubed
        
    return Ex, Ey, Ez

# --- 3. Plot 1: 2D Static Streamplot (X-Y Plane) ---

def plot_2d_static():
    print("Generating 2D static plot...")
    # Create a 2D grid in the x-y plane (Z=0)
    nx, ny = 50, 50
    lim = 3
    x = np.linspace(-lim, lim, nx)
    y = np.linspace(-lim, lim, ny)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)  # 2D slice at z=0

    # Calculate E-field on this 2D grid
    Ex, Ey, Ez = get_E_grid(charge_list, X, Y, Z)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Calculate magnitude for coloring and line width
    E_mag = np.sqrt(Ex**2 + Ey**2)
    # Use log scale for better visualization of field strength
    log_E = np.log10(E_mag) 

    # Plot streamplot
    strm = ax.streamplot(X, Y, Ex, Ey, 
                         color=log_E, 
                         cmap='viridis', 
                         linewidth=1.5, 
                         density=1.5, 
                         arrowstyle='->', 
                         arrowsize=1.2)
    
    # Add a colorbar
    fig.colorbar(strm.lines, ax=ax, label='log10(|E|)')

    # Plot charges
    for q, (xq, yq, zq) in charge_list:
        color = 'red' if q > 0 else 'blue'
        ax.plot(xq, yq, 'o', markersize=12, color=color, 
                markeredgecolor='white', markeredgewidth=1.5)

    ax.set_xlabel('X Position', fontsize=12)
    ax.set_ylabel('Y Position', fontsize=12)
    ax.set_title('2D Electric Field Streamplot', fontsize=16, weight='bold')
    ax.set_aspect('equal')
    ax.set_xlim([-lim, lim])
    ax.set_ylim([-lim, lim])
    plt.show()

# --- 4. Plot 2: 3D Static Quiver Plot ---

def plot_3d_static():
    print("Generating 3D static plot...")
    # Create a sparse 3D grid
    n = 10  # 10x10x10 grid points
    lim = 3
    # Use mgrid for a simple 3D grid
    X, Y, Z = np.mgrid[-lim:lim:n*1j, -lim:lim:n*1j, -lim:lim:n*1j]
    
    # Get E-field
    Ex, Ey, Ez = get_E_grid(charge_list, X, Y, Z)
    
    # Normalize vectors for better visualization (we care about direction)
    E_mag = np.sqrt(Ex**2 + Ey**2 + Ez**2)
    E_mag[E_mag == 0] = 1e-6  # Avoid division by zero
    Ex_n = Ex / E_mag
    Ey_n = Ey / E_mag
    Ez_n = Ez / E_mag
    
    # Plotting
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the 3D quiver
    # We use E_mag as the color to show strength
    colors = E_mag.flatten()
    # Normalize colors for the colormap
    colors = (colors - colors.min()) / (colors.max() - colors.min())
    
    ax.quiver(X, Y, Z, 
              Ex_n, Ey_n, Ez_n, 
              length=0.5,  # Fixed arrow length
              normalize=True, 
              arrow_length_ratio=0.4,
              colors=plt.cm.viridis(colors))
    
    # Plot charges
    for q, (xq, yq, zq) in charge_list:
        color = 'red' if q > 0 else 'blue'
        ax.plot([xq], [yq], [zq], 'o', markersize=15, color=color,
                markeredgecolor='black', markeredgewidth=1.5)

    ax.set_xlabel('X Axis', fontsize=12)
    ax.set_ylabel('Y Axis', fontsize=12)
    ax.set_zlabel('Z Axis', fontsize=12)
    ax.set_title('3D Electric Field Quiver Plot', fontsize=16, weight='bold')
    plt.show()

# --- 5. Plot 3: 2D Animation (Rotating Dipole) ---

def plot_2d_animation():
    print("Generating 2D animation... (This may take a moment)")
    
    # Setup the figure
    fig_anim, ax_anim = plt.subplots(figsize=(8, 8))
    
    # Create a 2D grid (sparse for faster animation)
    nx_anim, ny_anim = 20, 20
    lim = 3
    x_anim = np.linspace(-lim, lim, nx_anim)
    y_anim = np.linspace(-lim, lim, ny_anim)
    X_anim, Y_anim = np.meshgrid(x_anim, y_anim)
    Z_anim = np.zeros_like(X_anim)
    
    # Original dipole configuration (q and d from the top)
    
    def update(frame):
        # Clear the previous plot
        ax_anim.cla()
        
        # Calculate new charge positions for a rotating dipole
        # Rotate by 4 degrees per frame
        angle = np.radians(frame * 4)
        
        xq_pos = (d / 2) * np.cos(angle)
        yq_pos = (d / 2) * np.sin(angle)
        
        xq_neg = -(d / 2) * np.cos(angle)
        yq_neg = -(d / 2) * np.sin(angle)
        
        anim_charges = [
            (q, (xq_pos, yq_pos, 0)),
            (-q, (xq_neg, yq_neg, 0))
        ]
        
        # Recalculate E-field
        Ex, Ey, Ez = get_E_grid(anim_charges, X_anim, Y_anim, Z_anim)
        
        # Get magnitude and normalize (for quiver plot)
        E_mag = np.sqrt(Ex**2 + Ey**2)
        E_mag[E_mag == 0] = 1e-6
        Ex_n = Ex / E_mag
        Ey_n = Ey / E_mag
        
        # Plot quiver (faster than streamplot for animation)
        # Color arrows by magnitude
        ax_anim.quiver(X_anim, Y_anim, Ex_n, Ey_n, E_mag, 
                       cmap='coolwarm', pivot='middle', scale=30)
        
        # Plot charges
        for q_val, (xq, yq, zq) in anim_charges:
            color = 'red' if q_val > 0 else 'blue'
            ax_anim.plot(xq, yq, 'o', markersize=12, color=color,
                         markeredgecolor='white', markeredgewidth=1.5)

        # Set plot properties for each frame
        ax_anim.set_xlabel('X Position')
        ax_anim.set_ylabel('Y Position')
        ax_anim.set_title(f'Animated Rotating Dipole (Angle: {frame*4}°)', 
                          fontsize=14, weight='bold')
        ax_anim.set_aspect('equal')
        ax_anim.set_xlim([-lim, lim])
        ax_anim.set_ylim([-lim, lim])
        
    # Create animation
    # frames=90 for a full 360-degree rotation (90 * 4 = 360)
    ani = FuncAnimation(fig_anim, update, frames=90, interval=100, blit=False)
    
    plt.show()
    # Note: To save the animation as a .gif (requires 'imagemagick'):
    # ani.save('dipole_animation.gif', writer='imagemagick', fps=10)

# --- Main execution ---
if __name__ == "__main__":
    plot_2d_static()
    plot_3d_static()
    plot_2d_animation()